# **💁🏻🗨️💁🏻‍♂️안개 예측 EDA code**
> **안개량 예측** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최대 10명이 참여할 수 있는 기상청 주관 날씨 빅데이터 경진대회 입니다.     
> 주어진 데이터를 활용하여 안개 상태의 구간을 예측할 수 있는 모델을 만드는 것이 목표입니다!

# Contents  
  
- 필요한 라이브러리 설치  
- 데이터 불러오기  
- 데이터 결측치 처리하기
- 모델 적합하기
- 추론하기
- 제출물 만들기

완전히 초기화 되어있는 상태에서 작성하였기 때문에 단순하게 적용한 방법에서 변경해 나갈 예정

### 1. 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [84]:
# basic
import os, random
import pandas as pd
import numpy as np

# model
from sklearn.tree import DecisionTreeClassifier

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [85]:
# random seed 고정하기
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

### 2. 데이터 불러오기
- 제공된 데이터를 불러옵니다.

> - year : 년도
> - month : 월
> - day : 일
> - hour : 시간
> - minute : 분(10분 단위)
> - stn_id : 지점 번호
> - ws10_deg : 10분 평균 풍향, deg
> - ws10_ms : 10분 평균 풍속, m/s
> - ta : 1분 평균 기온 10분 주기, 섭씨
> - re : 강수 유무 0:무강수, 1:강수
> - hm : 1분 평균 상대 습도 10분 주기, %
> - sun10 : 1분 일사량 10분 단위 합계, MJ
> - ts : 1분 평균 지면온도 10분 주기, 섭씨

- test 없는 데이터 값
> - vis1 : 1분 평균 시정 10분 주기, m
> - class : 시정 구간

시정 구간은 다음과 같다.
- 0초과 200미만 : 1
- 200이상 500미만 : 2
- 500이상 1000미만 : 3
- 1000이상 : 4
- 4번은 맞춰도 스코어가 증가하진 않지만 틀리면 감점

In [86]:
# load makes data
train = pd.read_csv('../data/fog_train.csv')
test = pd.read_csv('../data/fog_test.csv')

In [87]:
# 쓸데없는 열 제거
train.drop(['Unnamed: 0'], axis = 1, inplace = True)
test.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [88]:
# 결측치 nan으로 바꾸기
train2 = train.replace(-99, np.nan).replace(-99.9, np.nan).replace(-999.0, np.nan)
test2 = test.replace(-99, np.nan).replace(-99.9, np.nan).replace(-999.0, np.nan)

In [89]:
# 변수 이름 앞에있는 이상한거 제거하기
train2.columns = train2.columns.str.replace('fog_train.', '')
test2.columns = test2.columns.str.replace('fog_test.', '')

### 3. 결측값 대치하기

- 단순하게 결측값 대치하기: 각각 중앙값으로 대치

In [90]:
# 대치할 중위수
med_train = train2.median()
med_train.index

Index(['month', 'day', 'time', 'minute', 'ws10_deg', 'ws10_ms', 'ta', 're',
       'hm', 'sun10', 'ts', 'vis1', 'class'],
      dtype='object')

In [91]:
# 결측치 채우기
for c in med_train.index:
    if c == 'vis1':
        continue
    train2[c][train2[c].isna()] = med_train[c]
    test2[c][test2[c].isna()] = med_train[c]

### 4. 모델 적합하기

- 간단하게 decision tree를 통해 baseline을 형성하고자 한다.
- class로 분해를 적용할 예정

In [92]:
# label 선택하기
use_label_x = ['month', 'time', 'ws10_deg', 'ws10_ms', 'ta', 're',
       'hm', 'sun10', 'ts']

use_label_y = ['class']

In [93]:
# train_x, train_y, test_x 만들자
train_x = train2[use_label_x]
train_y = train2[use_label_y]

test_x = test2[use_label_x]

In [94]:
# model
dt = DecisionTreeClassifier()

In [95]:
# train
dt.fit(train_x, train_y)

DecisionTreeClassifier()

### 5. 추론하기

- 대입하여 추론하기

In [96]:
pred = dt.predict(test_x)

### 6. 결과물 출력하기

- class에 대해 두가지 버전으로 출력할 예정
1. 결측치 아닌것은 제외시키기
2. 결측치까지 모두 대입해주기

In [97]:
# 복제
test_submit1 = test.copy()
test_submit2 = test.copy()

In [98]:
# 1. 모든 값에 대해 대입하기
test_submit1['fog_test.class'] = pred

In [99]:
# 결측치가 아닌것에 대입하기
test_submit2['fog_test.class'] = np.where(test_submit2['fog_test.class'].isna(), pred, test_submit2['fog_test.class'])

In [106]:
# 저장하기
test_submit1.to_csv('../Data/submission_모든값.csv', index = False)
test_submit2.to_csv('../Data/submission_-99제외.csv', index = False)
# 제출시 다음과 같은 이름으로 적용해야함
# test_submit1.to_csv('../Data/240253.csv', index = False)

In [102]:
test_submit2['fog_test.class'].value_counts()

 4.0     256752
-99.0      2694
 3.0       1490
 2.0       1105
 1.0        759
Name: fog_test.class, dtype: int64

In [103]:
test_submit1['fog_test.class'].value_counts()

4.0    259398
3.0      1513
2.0      1115
1.0       774
Name: fog_test.class, dtype: int64